In [49]:
import pandas as pd
import time
import os
from google import genai
import json
from dotenv import load_dotenv

In [ ]:
load_dotenv()
client = genai.Client(api_key=os.getenv("api_key"))

In [ ]:
df = pd.read_excel("source data/database cbo.xlsx", sheet_name = 0)
unique_cbo = df['cbo'].unique()

In [ ]:
for cbo_atual in unique_cbo:
    print(f"\n🔁 Processando CBO: {cbo_atual}")

    df_filtrado = df[df['cbo'] == cbo_atual].copy()
    df_filtrado['task_full'] = df_filtrado['task_group'] + ': ' + df_filtrado['task']
    tabela_tarefas = df_filtrado[['task_full', 'key']].to_string(index=False)

    description = df_filtrado['description'].unique()

    prompt_usuario = f"""
Você é um especialista em análise ocupacional e economia do trabalho, com profundo conhecimento da Classificação Brasileira de Ocupações (CBO).

Sua tarefa é avaliar criticamente a *Frequência* e a *Importância* de uma lista de tarefas para o desempenho eficaz da ocupação "{description}", identificada pelo CBO {cbo_atual}.

*Instrução Mestra: O Princípio da Centralidade Funcional*

Antes de avaliar qualquer tarefa, você deve primeiro considerar o *núcleo funcional* da ocupação "{description}". Responda mentalmente: "Qual é o propósito principal e a entrega de valor fundamental deste cargo?". Esta resposta será sua âncora para todas as avaliações.

- *Contexto Acima de Tudo:* A avaliação de cada tarefa deve ser relativa à sua contribuição para o núcleo funcional da ocupação.
- *Evite o Viés de Competências Genéricas:* Competências pessoais (ex: "Demonstrar Empatia", "Trabalhar em Equipe") devem ser pontuadas estritamente dentro do contexto da ocupação.
    - *Exemplo-Guia:* A tarefa "Demonstrar Empatia" é *extremamente importante (5)* e *constante (5)* para um Psicanalista (CBO 2515-50), pois é a ferramenta central de seu trabalho. Para um Policial Militar (CBO 5172-10), a mesma tarefa, embora útil, é *ocasional (2)* e de *importância moderada (3)*, pois o núcleo funcional do cargo é a aplicação da lei e a manutenção da ordem.
- *Análise Global:* Analise a lista completa de tarefas antes de pontuar, para entender o escopo geral e garantir uma distribuição de notas coerente e diferenciada.

*Critérios de Avaliação*

1.  *Frequência (Escala de 1 a 5):* Quão prevalente é a tarefa na rotina de trabalho?
    - *1 (Muito Raramente):* Atividade excepcional, não faz parte da rotina esperada do cargo.
    - *2 (Ocasionalmente):* Acontece de forma pontual ou em resposta a situações específicas; não é uma atividade programada ou rotineira.
    - *3 (Frequentemente):* Parte regular e esperada das responsabilidades, mas não ocupa a maior parte do tempo de trabalho.
    - *4 (Muito Frequentemente):* Uma das principais atividades do cargo; ocupa uma porção significativa do tempo e da atenção diária.
    - *5 (Constantemente / Tarefa Central):* É a essência da ocupação. A atividade que define o papel e é realizada de forma quase contínua.

2.  *Importância (Escala de 1 a 5):* Qual o impacto da tarefa para o sucesso e o propósito da ocupação?
    - *1 (Nada Importante):* Tarefa acessória, sem impacto direto no resultado.
    - *2 (Pouco Importante):* Contribui marginalmente.
    - *3 (Moderadamente Importante):* Tarefa de suporte relevante, mas não crítica.
    - *4 (Muito Importante):* Crítica para o bom desempenho, mas não é a missão principal.
    - *5 (Extremamente Importante):* Tarefa fundamental. Sua má execução compromete diretamente a missão e os resultados essenciais do cargo.

*Formato da Resposta*

Gere a frequência e a importância para cada tarefa na tabela abaixo. Sua resposta deve ser um JSON limpo, sem nenhum texto ou justificativa adicional.

*Ocupação:* "{description}"
*CBO:* {cbo_atual}
*Tarefas para Avaliação:*
{tabela_tarefas}

*JSON de Saída:*
[
  {{
    "key": "key",
    "frequency": NUMERO_DE_1_A_5,
    "importance": NUMERO_DE_1_A_5
  }},
  {{
    "key": "key",
    "frequency": NUMERO_DE_1_A_5,
    "importance": NUMERO_DE_1_A_5
  }}
]
"""

    try:
        tempo_inicio = time.time()

        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=prompt_usuario,
        )
        resposta = response.text
        resposta = resposta.replace("```json", "").replace("```", "")
        tempo_fim = time.time()
        duracao = tempo_fim - tempo_inicio
        print(f"CBO {cbo_atual} processado em {duracao:.2f} segundos")

        arquivo_saida = os.path.join("json/frequency_importance", f"{cbo_atual}.json")
        with open(arquivo_saida, "w", encoding="utf-8") as f:
            json.dump(json.loads(resposta), f, ensure_ascii=False, indent=4)

    except Exception as e:
        print(f"Erro ao processar CBO {cbo_atual}: {e}")

In [53]:
all_json = []
arquivos_com_erro = []

pasta = "json/frequency_importance"

for nome_arquivo in os.listdir(pasta):
    if nome_arquivo.endswith(".json"):
        caminho_arquivo = os.path.join(pasta, nome_arquivo)
        try:
            with open(caminho_arquivo, "r", encoding="utf-8") as f:
                conteudo = f.read().strip()
                if not conteudo:
                    raise ValueError("Arquivo vazio")
                dados = json.loads(conteudo)
                if isinstance(dados, list):
                    all_json.extend(dados)
                else:
                    all_json.append(dados)
        except Exception as e:
            print(f"Erro no arquivo '{nome_arquivo}': {e}")
            arquivos_com_erro.append(caminho_arquivo)


for caminho in arquivos_com_erro:
    try:
        os.remove(caminho)
        print(f"Arquivo deletado: {os.path.basename(caminho)}")
    except Exception as e:
        print(f"Erro ao deletar {os.path.basename(caminho)}: {e}")


with open("json/frequency_importance.json", "w", encoding="utf-8") as f:
    json.dump(all_json, f, ensure_ascii=False, indent=2)

In [54]:
with open("json/frequency_importance.json", "r", encoding="utf-8") as f:
    data = json.load(f)

df = pd.DataFrame(data)

df.to_excel("results/frequency_importance.xlsx", index=False)